# Hypotheses 3 and 4 - Data Wrangling 

### Set up everything

In [4]:
# Set up
import pandas as pd
from sodapy import Socrata
import requests
from pprint import pprint
import gmaps
import os
import json

from api_keys_tokens import MyAppToken
from api_keys_tokens import google_key

### Step 1: Get the data from the Pedestrian Counting System - Monthly (counts per hour) database

#### Make an API call

In [5]:
# Pedestrian Counting System - Monthly (counts per hour)
# API Documentation - https://dev.socrata.com/foundry/data.melbourne.vic.gov.au/b2ak-trbp

client = Socrata("data.melbourne.vic.gov.au", MyAppToken )
pedestrian_count_dataset_id = "b2ak-trbp"
pedestrian_count_request = client.get(pedestrian_count_dataset_id, limit = 4000000)

# Assign it to a data frame
pedestrian_count_df = pd.DataFrame.from_records(pedestrian_count_request)

#### Save as a csv file

In [6]:
# Save as a csv file
pedestrian_count_df.to_csv("Melbourne City Council Data/original data/all_pedestrian_data.csv",encoding = 'utf-8', index = False)

#### Make a CSV file containing only pedestrian data from August 2019 to August 2021

In [7]:
# Read the data
data = pd.read_csv("Melbourne City Council Data/original data/all_pedestrian_data.csv")

# Find only the data between the beginning of August 2019 and the end of August 2021
filtered_df = data.loc[(data['date_time'] >= '2019-08-01') & (data['date_time'] < '2021-09-01')].sort_values(["id"])

# Save result as a csv
filtered_df.to_csv("Melbourne City Council Data/filtered_pedestrian_data.csv",encoding = 'utf-8', index = False)

### Step 2: Get all the sensor location data via an API call

#### Make an API call

In [8]:
# Sensor Locations
# API Documentation - https://dev.socrata.com/foundry/data.melbourne.vic.gov.au/h57g-5234
client = Socrata("data.melbourne.vic.gov.au", MyAppToken )
sensor_location_id = "h57g-5234"
sensor_location_id_request = client.get(sensor_location_id, limit = 100)

# Convert to pandas DataFrame
sensor_location_df = pd.DataFrame.from_records(sensor_location_id_request)

#### Save as a csv file

In [9]:
# Save as a csv file
sensor_location_df.to_csv("Melbourne City Council Data/original data/all_sensor_location_data.csv",encoding = 'utf-8', index = False)

### Step 3: Find the Postcode for Each Sensor  (using Google API Mapping)

In [10]:
# Get the data
data = pd.read_csv("Melbourne City Council Data/filtered_pedestrian_data.csv")
sensor_data = pd.read_csv("Melbourne City Council Data/original data/all_sensor_location_data.csv")

# Add a "Postcode" column to the DataFrame.
sensor_data["Address"] = ""
# base url
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

# use iterrows to iterate through pandas dataframe
for index, row in sensor_data.iterrows():
    
    # Set the parameters of the search
    params = {
    "latlng": f"{sensor_data.loc[index, 'latitude']}, {sensor_data.loc[index, 'longitude']}",
    "key": google_key
    }
    
    # assemble url and make API request
    try: 
        response = requests.get(base_url, params = params).json()
        # Get the postcode
        address = response["results"][0]["formatted_address"]
        # Assign it to the hotel DataFrame
        sensor_data.loc[index, 'Address'] = address
        # Log the result
        print(address)
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("-----------------------------------------------------")


380A Elizabeth St, Melbourne VIC 3000, Australia
-----------------------------------------------------
150 Swanston St, Melbourne VIC 3000, Australia
-----------------------------------------------------
418 Swanston St, Melbourne VIC 3000, Australia
-----------------------------------------------------
Unnamed Road, Melbourne VIC 3004, Australia
-----------------------------------------------------
123 Flinders St, Melbourne VIC 3000, Australia
-----------------------------------------------------
252 La Trobe St, Melbourne VIC 3000, Australia
-----------------------------------------------------
231 Bourke Street Mall, Melbourne VIC 3000, Australia
-----------------------------------------------------
Sandridge Bridge, 1 Southbank Blvd, Southbank VIC 3006, Australia
-----------------------------------------------------
703/37 Swanston St, Melbourne VIC 3000, Australia
-----------------------------------------------------
2 Clarendon St, South Wharf VIC 3006, Australia
---------------

#### Extract the postcode from the address

In [11]:
sensor_data["Postcode"] = sensor_data["Address"].str.slice(start = -15,stop = -11)

#### Save a copy of the sensor data containing the postcode

In [12]:
sensor_data.to_csv("Melbourne City Council Data/sensor_data_postcode.csv")

### Step 4: Extract a list of all the unique postocdes found in the pedestrian sensor data.

In [13]:
# Read the data
sensor_data_postcode = pd.read_csv("Melbourne City Council Data/sensor_data_postcode.csv")
# Get all the unique postcodes
postcode_list = sensor_data_postcode["Postcode"].unique()

### Step 5: Sum up the pedestrian counts per sensor per month for the periof of August 2019 to August 2021

#### Sum the data

In [14]:
# Read the data
pedestrian_data = pd.read_csv("Melbourne City Council Data/filtered_pedestrian_data.csv")

# Sum the data by year, month, and sensor id
summed_pedestrian_data = pedestrian_data.groupby(["year","month","sensor_id"]).sum("hourly_counts")
summed_pedestrian_data = summed_pedestrian_data["hourly_counts"].reset_index()

summed_pedestrian_data["month-year"] = summed_pedestrian_data["month"].astype(str) + " " + summed_pedestrian_data["year"].astype(str)
summed_pedestrian_data = summed_pedestrian_data.drop(["year","month"], axis = 1)
# rename hourly count column
summed_pedestrian_data = summed_pedestrian_data.rename(columns = {"hourly_counts": "monthly_count"})

ped_df = summed_pedestrian_data.pivot(index = "sensor_id", columns = "month-year", values = "monthly_count")
ped_df

month-year,April 2020,April 2021,August 2019,August 2020,August 2021,December 2019,December 2020,February 2020,February 2021,January 2020,...,March 2020,March 2021,May 2020,May 2021,November 2019,November 2020,October 2019,October 2020,September 2019,September 2020
sensor_id,,,,,,,,,,,,,,,,,,,,,
1,118072.0,612164.0,838069.0,70551.0,156531.0,1192463.0,714656.0,799619.0,354240.0,892048.0,...,560956.0,523660.0,269797.0,NaN,1037267.0,498611.0,1028681.0,155212.0,943758.0,71264.0
2,90403.0,361785.0,641791.0,55197.0,80548.0,927155.0,512225.0,576590.0,215505.0,668884.0,...,413673.0,315709.0,166652.0,NaN,744945.0,299204.0,477010.0,99598.0,643780.0,53963.0
3,127245.0,337429.0,981893.0,92864.0,105047.0,911297.0,351662.0,741496.0,104829.0,761665.0,...,560572.0,210303.0,221395.0,280760.0,901523.0,425215.0,919197.0,200440.0,869674.0,114151.0
4,130659.0,743653.0,1064689.0,97813.0,213065.0,1244392.0,796734.0,905988.0,456419.0,1030896.0,...,634135.0,650570.0,245937.0,611918.0,1074747.0,490630.0,1047186.0,198010.0,971314.0,112765.0
5,98688.0,759926.0,963378.0,91977.0,178661.0,1068849.0,731645.0,942341.0,515746.0,1247383.0,...,595265.0,660726.0,156405.0,NaN,673975.0,364502.0,1028104.0,184081.0,925437.0,112820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,9753.0,NaN,NaN,NaN,NaN,58787.0,NaN,64887.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,NaN,32005.0,NaN,NaN,15795.0,NaN,NaN,NaN,23649.0,NaN,...,NaN,30935.0,NaN,31189.0,NaN,NaN,NaN,NaN,NaN,NaN
76,NaN,41175.0,NaN,NaN,42283.0,NaN,NaN,NaN,NaN,NaN,...,NaN,45657.0,NaN,45442.0,NaN,NaN,NaN,NaN,NaN,NaN


#### Clean the data ( Remove all Null Values)

In [15]:
# Count NA values in each column
ped_df.isnull().sum(axis = 1)

#Remove all rows with NA values in them
ped_df = ped_df.dropna()
# Remove the extra columns
ped_df = ped_df.reset_index()
ped_df = ped_df.rename_axis(None, axis=1)

#### Create a CSV file of the summed values

In [16]:
# Send to csv
ped_df.to_csv("Melbourne City Council Data/summed_pedestrian_count_sensor_Aug19Aug20.csv",index = False)

### Step 6: Merge summed pedestrian count per sensor per month with sensor location data

#### Merge the data

In [17]:
# Get sensor location data
sensor_data = pd.read_csv("Melbourne City Council Data/sensor_data_postcode.csv")

# Merge it with the other summed Pedestrian values data.
pedestrian_location_sensor_data = ped_df.merge(sensor_data, on="sensor_id")
pedestrian_location_sensor_data

# Remove unwanted columns
pedestrian_location_sensor_data = pedestrian_location_sensor_data.drop(columns=["installation_date","location","note"])
del pedestrian_location_sensor_data["Unnamed: 0"]

#### Create a CSV file of the merged data

In [18]:
# Save to csv file
pedestrian_location_sensor_data.to_csv("Melbourne City Council Data/pedestrian_location_sensor_data.csv", index = False)

### Step 7: Find Pedestrian Activity by Postcode per month

#### Group Pedestrian activity by postcode

In [19]:
# Sum the pedestrian traffic by postcode
pedestrian_location_sensor_data_sum = pedestrian_location_sensor_data.groupby("Postcode").sum()

# Get rid of unwanted columns
pedestrian_location_sensor_data_sum = pedestrian_location_sensor_data_sum.drop(columns = ["sensor_id","latitude","longitude"])

#  Show results
pedestrian_location_sensor_data_sum

,April 2020,April 2021,August 2019,August 2020,August 2021,December 2019,December 2020,February 2020,February 2021,January 2020,...,March 2020,March 2021,May 2020,May 2021,November 2019,November 2020,October 2019,October 2020,September 2019,September 2020
Postcode,,,,,,,,,,,,,,,,,,,,,
3000,1024479.0,3915301.0,6933852.0,781393.0,1313082.0,6661593.0,3476653.0,5735095.0,2316395.0,5884819.0,...,4237348.0,3480340.0,1622996.0,3454668.0,6325968.0,2618461.0,6855644.0,1285169.0,6287155.0,882223.0
3004,109479.0,629414.0,925192.0,102140.0,154748.0,819130.0,367854.0,765504.0,369535.0,994735.0,...,464896.0,522324.0,153313.0,479879.0,866647.0,221087.0,900504.0,150489.0,878428.0,118878.0
3006,41299.0,121408.0,245551.0,34641.0,54388.0,279113.0,45554.0,190047.0,23016.0,279641.0,...,244703.0,150365.0,63847.0,96813.0,323731.0,75898.0,306072.0,57310.0,239907.0,45078.0
3008,114180.0,521195.0,1034717.0,115428.0,182531.0,582828.0,227975.0,814956.0,162129.0,620898.0,...,478396.0,336060.0,151740.0,468822.0,721514.0,158432.0,595880.0,151652.0,659475.0,133627.0
3010,26338.0,112208.0,356111.0,26930.0,49191.0,145925.0,59427.0,187187.0,73307.0,139888.0,...,222884.0,142908.0,36920.0,117954.0,210204.0,71125.0,263562.0,48637.0,274422.0,31362.0
3052,11460.0,85887.0,217817.0,13801.0,26917.0,50258.0,22905.0,99105.0,38200.0,64079.0,...,143196.0,101222.0,20132.0,105440.0,85326.0,27380.0,134082.0,22574.0,149689.0,17839.0
3053,117199.0,405995.0,537900.0,108934.0,158427.0,563859.0,456104.0,503761.0,295097.0,537851.0,...,395306.0,392560.0,174678.0,351161.0,574188.0,428256.0,579677.0,196227.0,525270.0,132336.0


#### Fix the format of the data

In [20]:
# Sort the values of the columns
pedestrian_location_sensor_data_sum = pedestrian_location_sensor_data_sum[['August 2019', 'September 2019',
                                                                           'October 2019', 'November 2019', 'December 2019',
                                                                           'January 2020', 'February 2020', 'March 2020',
                                                                           'April 2020', 'May 2020', 'June 2020',
                                                                           'July 2020', 'August 2020', 'September 2020',
                                                                           'October 2020', 'November 2020', 'December 2020', 
                                                                           'January 2021','February 2021', 'March 2021',
                                                                           'April 2021', 'May 2021', 'June 2021',
                                                                           'July 2021', 'August 2021'
                                                                          ]]
# Transpose the data for mapping
df = pedestrian_location_sensor_data_sum.T

# Fix the names and columns
df = df.reset_index()
df = df.rename(columns = {"index": "Date"})
# Fix the name of the index
df = df.rename_axis(None, axis = 1)

# Show result
df

,Date,3000,3004,3006,3008,3010,3052,3053
0,August 2019,6933852.0,925192.0,245551.0,1034717.0,356111.0,217817.0,537900.0
1,September 2019,6287155.0,878428.0,239907.0,659475.0,274422.0,149689.0,525270.0
2,October 2019,6855644.0,900504.0,306072.0,595880.0,263562.0,134082.0,579677.0
3,November 2019,6325968.0,866647.0,323731.0,721514.0,210204.0,85326.0,574188.0
4,December 2019,6661593.0,819130.0,279113.0,582828.0,145925.0,50258.0,563859.0
5,January 2020,5884819.0,994735.0,279641.0,620898.0,139888.0,64079.0,537851.0
6,February 2020,5735095.0,765504.0,190047.0,814956.0,187187.0,99105.0,503761.0
7,March 2020,4237348.0,464896.0,244703.0,478396.0,222884.0,143196.0,395306.0
8,April 2020,1024479.0,109479.0,41299.0,114180.0,26338.0,11460.0,117199.0
9,May 2020,1622996.0,153313.0,63847.0,151740.0,36920.0,20132.0,174678.0


#### Save as a CSV File

In [21]:
# Save as a csv
df.to_csv("Melbourne City Council Data/Pedestrian_Activity_by_Postcode_per_month.csv", index = False)

### Step 8: Find Pedestrian activity shortly after the end of JobKeeper (April 2021 upto and including August 2021)

#### Filter the data

In [22]:
# Get the data
data = pd.read_csv("Melbourne City Council Data/Pedestrian_Activity_by_Postcode_per_month.csv")
# Filter the data
after_jobkeeper = data.loc[(data["Date"] == "April 2021") |
                           (data["Date"] == "May 2021") |
                           (data["Date"] == "June 2021") |
                           (data["Date"] == "July 2021") |
                           (data["Date"] == "August 2021")
                           ]
# Show result
after_jobkeeper

,Date,3000,3004,3006,3008,3010,3052,3053
20,April 2021,3915301.0,629414.0,121408.0,521195.0,112208.0,85887.0,405995.0
21,May 2021,3454668.0,479879.0,96813.0,468822.0,117954.0,105440.0,351161.0
22,June 2021,2242169.0,300066.0,51259.0,210340.0,55170.0,36669.0,281944.0
23,July 2021,2513881.0,469896.0,91690.0,278798.0,62019.0,42580.0,300682.0
24,August 2021,1313082.0,154748.0,54388.0,182531.0,49191.0,26917.0,158427.0


#### Save as a CSV File

In [23]:
# Send to a csv
after_jobkeeper.to_csv("Melbourne City Council Data/after_jobkeeper_pedestrian_count.csv", index = False)

### Step 9: Pedestrian activity before the beginning of JobKeeper (August 2019 until the end of March 2020)

#### Filter the data

In [24]:
# Get the data
data = pd.read_csv("Melbourne City Council Data/Pedestrian_Activity_by_Postcode_per_month.csv")
# Filter the data
before_jobkeeper = data.loc[(data["Date"] == "August 2019") |
                           (data["Date"] == "September 2019") |
                           (data["Date"] == "October 2019") |
                           (data["Date"] == "November 2019") |
                           (data["Date"] == "December 2019") |
                            (data["Date"] == "January 2020") |
                            (data["Date"] == "February 2020") |
                            (data["Date"] == "March 2020")
                           ]

#### Save as a CSV File

In [25]:
# Send to a csv
before_jobkeeper.to_csv("Melbourne City Council Data/before_jobkeeper_pedestrian_count.csv", index=False)

### Step 10: Pedestrian Traffic during JobKeeper

#### Filter the data

In [26]:
# Get the data
data = pd.read_csv("Melbourne City Council Data/Pedestrian_Activity_by_Postcode_per_month.csv")

# Create a list of all the JobKeeper Dates
date_list = ['April 2020', 'May 2020', 'June 2020', 'July 2020', 'August 2020', 'September 2020', 
             'October 2020', 'November 2020', 'December 2020', 'January 2021','February 2021', 'March 2021']

# Filter the data in the the JobKeeper dates
jobkeeper_pedestrian_data = data.loc[(data["Date"].isin(date_list))]

#### Save as a CSV File

In [27]:
jobkeeper_pedestrian_data.to_csv("Melbourne City Council Data/during_jobkeeper_pedestrian_count.csv",index = False)

# JobKeeper Data

### Step 1: Filter JobKeeper data by the postcodes found in the Pedestrian Dataset

#### Split the data from 3 Excel spread sheets into 3 csv files

In [28]:
# Data Source: https://treasury.gov.au/coronavirus/jobkeeper/data

# Get the data
first_phase = pd.read_excel("jobkeeper data/original data/jobkeeperdata.xlsx", sheet_name = 'First Phase',header = 1, usecols = "A:G")
extension_quarter1 = pd.read_excel("jobkeeper data/original data/jobkeeperdata.xlsx", sheet_name = 'Extension - First Quarter',header = 1, usecols = "A:D")
extension_quarter2 = pd.read_excel("jobkeeper data/original data/jobkeeperdata.xlsx", sheet_name = 'Extension - Second Quarter',header = 1, usecols = "A:D")

# Save all of them to csv files
first_phase.to_csv("jobkeeper data/jkd_first_phase.csv", index = False)
extension_quarter1.to_csv("jobkeeper data/jkd_extension_quarter1.csv", index = False)
extension_quarter2.to_csv("jobkeeper data/jkd_extension_quarter2.csv", index = False)

#### Extract the data from the csv files and reformat them

In [29]:
# Get the data
first_phase_data = pd.read_csv("jobkeeper data/jkd_first_phase.csv")
first_extension_data = pd.read_csv("jobkeeper data/jkd_extension_quarter1.csv")
second_extension_data = pd.read_csv("jobkeeper data/jkd_extension_quarter2.csv")

# Convert the Postcodes of each dataFrame to strings
first_phase_data['Postcode'] = pd.Series(first_phase_data['Postcode'], dtype = pd.StringDtype())
first_extension_data['Postcode'] = pd.Series(first_extension_data['Postcode'], dtype = pd.StringDtype())
second_extension_data['Postcode'] = pd.Series(second_extension_data['Postcode'], dtype = pd.StringDtype())

# Cut the postcodes to the right length
first_phase_data['Postcode'] = first_phase_data['Postcode'].str.slice(start = 2)
first_extension_data['Postcode'] = first_extension_data['Postcode'].str.slice(start = 2)
second_extension_data['Postcode'] = second_extension_data['Postcode'].str.slice(start = 2)

#### Reformat them then merge them

In [30]:
# Get all the first phase data
MCC_first_phase_data = first_phase_data.loc[(first_phase_data["Postcode"] == "3000") |
                                            (first_phase_data["Postcode"] == "3004") |
                                            (first_phase_data["Postcode"] == "3006") |
                                            (first_phase_data["Postcode"] == "3008") |
                                            (first_phase_data["Postcode"] == "3010") |
                                            (first_phase_data["Postcode"] == "3031") |
                                            (first_phase_data["Postcode"] == "3051") |
                                            (first_phase_data["Postcode"] == "3052") |
                                            (first_phase_data["Postcode"] == "3053")
                                           ]

# Get all the first extension data
MCC_first_extension_data = first_extension_data.loc[
                                            (first_extension_data["Postcode"] == "3000") |
                                            (first_extension_data["Postcode"] == "3004") |
                                            (first_extension_data["Postcode"] == "3006") |
                                            (first_extension_data["Postcode"] == "3008") |
                                            (first_extension_data["Postcode"] == "3010") |
                                            (first_extension_data["Postcode"] == "3031") |
                                            (first_extension_data["Postcode"] == "3051") |
                                            (first_extension_data["Postcode"] == "3052") |
                                            (first_extension_data["Postcode"] == "3053")
                                           ]

# # Get all the second extension data
MCC_second_extension_data = second_extension_data.loc[(second_extension_data["Postcode"] == "3000") |
                                            (second_extension_data["Postcode"] == "3004") |
                                            (second_extension_data["Postcode"] == "3006") |
                                            (second_extension_data["Postcode"] == "3008") |
                                            (second_extension_data["Postcode"] == "3010") |
                                            (second_extension_data["Postcode"] == "3031") |
                                            (second_extension_data["Postcode"] == "3051") |
                                            (second_extension_data["Postcode"] == "3052") |
                                            (second_extension_data["Postcode"] == "3053")
                                           ]

## Merge the data frames
application_counts = MCC_first_phase_data.merge(MCC_first_extension_data, on="Postcode").merge(MCC_second_extension_data, on = "Postcode")

# Change names of the columns
application_counts = application_counts.rename(columns={"April Application Count": "April 2020",
                                                       "May Application Count": "May 2020",
                                                        "June Application Count": "June 2020",
                                                        "July Application Count": "July 2020",
                                                        "August Application Count": "August 2020",
                                                        "September Application Count": "September 2020",
                                                        "October Application Count": "October 2020",
                                                        "November Application Count": "November 2020",
                                                        "December Application Count": "Decemeber 2020",
                                                        "January Application Count": "January 2021",
                                                        "February Application Count": "February 2021",
                                                        "March Application Count": "March 2021"
                                                       })



#### Save as a new csv file

In [31]:
# Save the file to a csv
application_counts.to_csv("jobkeeper data/application_counts.csv", index = False)